In [131]:
%reset_selective -f vectorizerZ

In [65]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# load it again
with open('svm_with_higher_recall.pkl', 'rb') as fid:
    clf_loaded = cPickle.load(fid)

In [66]:
precision_recall_fscore_support(label_formatter(y_test),clf_loaded.predict(x_test))

(array([ 0.99783332,  0.87949401]),
 array([ 0.97269367,  0.98951311]),
 array([ 0.98510313,  0.93126542]),
 array([13257,  2670]))

In [67]:
precision_recall_fscore_support(label_formatter(y_val),clf_loaded.predict(x_val))

(array([ 0.99811352,  0.86502849]),
 array([ 0.96979598,  0.9906199 ]),
 array([ 0.98375101,  0.92357414]),
 array([12548,  2452]))

In [68]:
precision_recall_fscore_support(label_formatter(y_train),clf_loaded.predict(x_train))

(array([ 0.99795128,  0.87040844]),
 array([ 0.97109308,  0.98983637]),
 array([ 0.984339  ,  0.92628876]),
 array([50161,  9839]))

In [70]:
url  = "www.facebook.com/abhiagwl"

In [87]:
def url_to_input(url,model):
    x = hstack([complete_processor(Series(url))[0],vectorizer_spcl.transform(encoder(Series(url)).values),len_converter(Series(url))])
    output = model.predict(x)
    if (output[0]==1):
        print "Profile"
    else :
        print "Not Profile"

In [88]:
%time url_to_input(url,clf_loaded)

Profile
CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 11.8 ms


In [30]:
from nltk import ngrams
import pandas as pd
import numpy as np 
from pandas import Series
import pickle
import cPickle
import os
pd.options.display.max_rows=3000
root_dir = os.path.abspath('../..')
task_dir= os.path.join(root_dir,'user_profile')
data_dir=os.path.join(task_dir,'data')
final_data=os.path.join(data_dir,"final_scrap")
from sklearn import svm
from sklearn.utils import shuffle
from scipy.sparse import *
from sklearn.metrics import precision_recall_fscore_support

In [31]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [32]:
special_characters=['#','/', '%', '~', '=','?','&',"-","(",")","+","@","[","]","'","*"]

In [33]:
"loads the special character vectorizer"
with open('feature_genrator_special_charac.pkl', 'rb') as fid:
    vectorizer_spcl=cPickle.load(fid)

In [ ]:
with open('feature_genrator_vocab_.pkl', 'rb') as fid:
    vectorizerY=cPickle.load(fid)

In [34]:
"""Opens and stores the vocab for generating 3-gram features into vocab_ list for vectorizer input"""

the_filename="test_vocab"
with open(the_filename, 'rb') as f:
    vocab_ = pickle.load(f)

from sklearn.feature_extraction.text import CountVectorizer
vectorizerY= CountVectorizer(min_df=1)
vectorizerY.fit_transform(vocab_)
"""generating feature extraction for vocab"""

'generating vocab for feature extraction'

In [35]:
with open('feature_genrator_vocab_.pkl', 'wb') as fid:
    cPickle.dump(vectorizerY,fid)

In [36]:
import re
import urlparse
def hostname(x):
    return urlparse.urlparse(x).hostname

def path(x):
    return urlparse.urlparse(x).path

def preprocessor(list_):
    return list("".join(list_))

def gram_generator(x):
    grams = ngrams(x, 3)
    return ["".join(gram) for gram in grams]

def character_extracter(x):
    return x.str.lower().str.replace("http",".").str.replace("www",".").str.findall(r"\w+").apply(preprocessor)


def joined(list_):
    return " ".join(list_)

In [37]:
def encoder(x):
    x=x.str.replace("http://|https://","").str.findall("[^\:\w\.]").apply(joined)
    for n,spcl in enumerate(special_characters):
        x=x.str.replace(spcl,str(n+10))
    return x

In [38]:
def complete_processor(x,type_=1):
    x=x.iloc[np.random.permutation(x.shape[0])]
    x=character_extracter(x)
    x=x.apply(gram_generator).apply(joined).values
    x_=vectorizerY.transform(x)
    if type_==1:
        y_=csr_matrix(np.ones((x_.shape[0],1)))
    elif type_==0:
        y_=csr_matrix(np.zeros((x.shape[0],1)))

    return x_,y_

In [86]:
def len_converter(x):
    return csc_matrix(np.reshape(x.str.len().as_matrix(),(x.shape[0],1)))

In [17]:
def string_feature_gen(x):
    temp=x
    temp="".join(re.findall(r"\w+",x.lower().replace("http",".").replace("www",".")))
    temp=[" ".join(["".join(gram) for gram in ngrams(temp, 3)])]
    x_alpha_features=vec_alpha(temp)
    special_characters=['#','/', '%', '~', '=','?','&',"-","(",")","+","@","[","]","'","*"]
    temp=" ".join(re.findall("[^\:\w\.]",x.replace("http://|https://","")))
    for n,spcl in enumerate(special_characters):
        temp=temp.replace(spcl,str(n+10))
    x_special_features=vec_special.transform([temp])
    x_features=hstack([x_alpha_features,x_special_features,len(x)])

In [39]:
def label_formatter(y):
    return np.reshape(y.toarray(),(y.shape[0],))

In [55]:
#load the saved files
profile_urls=pd.read_csv(os.path.join(final_data,"profile_urls_from_working_data.csv"),header=None)[1]
non_profile_urls=pd.read_csv(os.path.join(final_data,"non_profile_urls_from_working_data.csv"),header=None)[1]

x_positive_len=len_converter(profile_urls)
x_negative_len=len_converter(non_profile_urls)

x_spcl_positive=vectorizer_spcl.transform(encoder(profile_urls).values)
x_spcl_negative=vectorizer_spcl.transform(encoder(non_profile_urls).values)

x_positive,y_positive=complete_processor(profile_urls,type_=1)
x_negative,y_negative=complete_processor(non_profile_urls,type_=0)

x_pos=hstack([x_positive,x_spcl_positive,x_positive_len])
x_neg=hstack([x_negative,x_spcl_negative,x_negative_len])

"this is to check the need for the handpicked feature apart from the fixed vocab"
x_pos=hstack([x_positive,x_positive_len])
x_neg=hstack([x_negative,x_negative_len])

x_pos=hstack([x_spcl_positive,x_positive_len])
x_neg=hstack([x_spcl_negative,x_negative_len])

In [61]:
np.random.seed(0) #to recreate results
x_data=vstack([x_neg,x_pos])
y_data=vstack([y_negative,y_positive])
x_data,y_data=shuffle(x_data,y_data)

In [62]:
x_train=x_data[:60000,:]
y_train=y_data[:60000,:]
x_val=x_data[60000:75000,:]
y_val=y_data[60000:75000,:]
x_test=x_data[75000:,:]
y_test=y_data[75000:,:]

In [ ]:
vec_alpha

In [63]:
with open('training_val_test_data', 'wb') as fid:
    cPickle.dump((x_train,y_train,x_val,y_val,x_test,y_test),fid)

In [ ]:
# load it again
with open('svm_with_higher_recall.pkl', 'rb') as fid:
    clf_loaded = cPickle.load(fid)

In [50]:
%who_ls

['CountVectorizer',
 'Series',
 'SparseEfficiencyWarning',
 'SparseWarning',
 'absolute_import',
 'base',
 'block_diag',
 'bmat',
 'bsr',
 'bsr_matrix',
 'cPickle',
 'character_extracter',
 'compressed',
 'construct',
 'coo',
 'coo_matrix',
 'cs_graph_components',
 'csc',
 'csc_matrix',
 'csgraph',
 'csr',
 'csr_matrix',
 'data',
 'data_dir',
 'dia',
 'dia_matrix',
 'diags',
 'division',
 'dok',
 'dok_matrix',
 'extract',
 'eye',
 'f',
 'fid',
 'final_data',
 'find',
 'gram',
 'gram_generator',
 'hostname',
 'hstack',
 'identity',
 'issparse',
 'isspmatrix',
 'isspmatrix_bsr',
 'isspmatrix_coo',
 'isspmatrix_csc',
 'isspmatrix_csr',
 'isspmatrix_dia',
 'isspmatrix_dok',
 'isspmatrix_lil',
 'joined',
 'kron',
 'kronsum',
 'lil',
 'lil_matrix',
 'load_sparse_csr',
 'n',
 'ngrams',
 'non_profile_urls',
 'np',
 'path',
 'pd',
 'pickle',
 'preprocessor',
 'print_function',
 'profile_urls',
 'rand',
 'random',
 're',
 'root_dir',
 'save_sparse_csr',
 'shuffle',
 'spcl',
 'spdiags',
 'special

In [ ]:
string_feature_gen(x)

In [66]:
precision_recall_fscore_support(label_formatter(y_test),clf_loaded.predict(x_test))

(array([ 0.99783332,  0.87949401]),
 array([ 0.97269367,  0.98951311]),
 array([ 0.98510313,  0.93126542]),
 array([13257,  2670]))

In [67]:
precision_recall_fscore_support(label_formatter(y_val),clf_loaded.predict(x_val))

(array([ 0.99811352,  0.86502849]),
 array([ 0.96979598,  0.9906199 ]),
 array([ 0.98375101,  0.92357414]),
 array([12548,  2452]))

In [40]:
url = "http://www.math.tamu.edu/~rowell/"

In [68]:
precision_recall_fscore_support(label_formatter(y_train),clf_loaded.predict(x_train))

(array([ 0.99795128,  0.87040844]),
 array([ 0.97109308,  0.98983637]),
 array([ 0.984339  ,  0.92628876]),
 array([50161,  9839]))

In [37]:
special_characters=['#','/', '%', '~', '=','?','&',"-","(",")","+","@","[","]","'","*"]

In [27]:
x=[" ".join(["".join(gram) for gram in ngrams(x, 3)])]

In [35]:
temp=" ".join(re.findall("[^\:\w\.]",x.replace("http://|https://","")))

In [38]:
for n,spcl in enumerate(special_characters):
        temp=temp.replace(spcl,str(n+10))

In [56]:
x_positive_len=csc_matrix(np.reshape(profile_urls.str.len().as_matrix(),(profile_urls.shape[0],1)))
x_negative_len=csc_matrix(np.reshape(non_profile_urls.str.len().as_matrix(),(non_profile_urls.shape[0],1)))

In [57]:
x_spcl_positive=vectorizer_spcl.transform(encoder(profile_urls).values)
x_spcl_negative=vectorizer_spcl.transform(encoder(non_profile_urls).values)

In [58]:
x_positive,y_positive=complete_processor(profile_urls,type_=1)
x_negative,y_negative=complete_processor(non_profile_urls,type_=0)

In [59]:
x_positive_len.shape

(14961, 1)

In [60]:
x_pos=hstack([x_positive,x_spcl_positive,x_positive_len])
x_neg=hstack([x_negative,x_spcl_negative,x_negative_len])

In [70]:
url  = "www.facebook.com/abhiagwl"

In [5]:
from scipy.sparse import *

In [238]:
x_pos=hstack([x_spcl_positive,x_positive_len])
x_neg=hstack([x_spcl_negative,x_negative_len])

In [87]:
def url_to_input(url,model):
    x = hstack([complete_processor(Series(url))[0],vectorizer_spcl.transform(encoder(Series(url)).values),len_converter(Series(url))])
    output = model.predict(x)
    if (output[0]==1):
        print "Profile"
    else :
        print "Not Profile"

In [88]:
%time url_to_input(url,clf_loaded)

Profile
CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 11.8 ms


In [286]:
clf=svm.LinearSVC(C=0.60).fit(x_train,label_formatter(y_train))

In [44]:
len(temp)

14

In [287]:
clf.score(x_val,label_formatter(y_val))

0.97666666666666668

In [288]:
clf.score(x_train,label_formatter(y_train))

0.9926666666666667

In [7]:
from sklearn import svm
from sklearn.utils import shuffle

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizerY= CountVectorizer(min_df=1)
vectorizerY.fit_transform(vocab_)
"""generating vocab for feature extraction"""

'generating vocab for feature extraction'

In [290]:
f1_score(label_formatter(y_val),clf.predict(x_val))

0.93052798729654618

In [277]:
clf.score(x_test,label_formatter(y_test))

0.978024737866516

In [54]:
x = hstack([csc_matrix(len(url)),vectorizer_spcl.transform(encoder(Series(url)).values),complete_processor(Series(url),type_=1)])

TypeError: invalid input format

In [50]:
x=x[:3]

In [53]:
csc_matrix(3)

<1x1 sparse matrix of type '<type 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Column format>

In [52]:
clf_loaded.predict(x)

ValueError: setting an array element with a sequence.

In [279]:
precision_score(label_formatter(y_train),clf.predict(x_train))

0.98113968734241053

In [73]:
x_1 = complete_processor(Series(url),type_=1)[0]

In [80]:
x_2 = vectorizer_spcl.transform(encoder(Series(url)).values)

In [77]:
x_3 = csc_matrix(len(url))

In [81]:
x_4  = hstack([x_1,x_2,x_3])

In [82]:
clf_loaded.predict(x_4)

array([ 1.])

In [281]:
recall_score(label_formatter(y_train),clf.predict(x_train))

0.98123865241073227

In [200]:
from sklearn.metrics import *

In [234]:

with open('svm_with_higher_recall.pkl', 'wb') as fid:
    cPickle.dump(clf, fid)    

In [78]:
x_3

<1x1 sparse matrix of type '<type 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Column format>

In [255]:
with open('feature_genrator_special_charac.pkl', 'wb') as fid:
    cPickle.dump(vectorizer_spcl,fid)

In [251]:
with open('feature_genrator.pkl', 'rb') as fid:
    vec_loaded=cPickle.load(fid)

x="".join(re.findall(r"\w+",x.lower().replace("http",".").replace("www",".")))

x=[" ".join(["".join(gram) for gram in ngrams(x, 3)])]

temp=" ".join(re.findall("[^\:\w\.]",x.replace("http://|https://","")))

for n,spcl in enumerate(special_characters):
        temp=temp.replace(spcl,str(n+10))

In [28]:
vectorizerY.transform(x)

<1x17576 sparse matrix of type '<type 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [ ]:
vectorizerY.transform([x])

In [ ]:
v

In [185]:
csc_matrix(np.reshape(profile_urls.str.len().as_matrix(),(profile_urls.shape[0],1)))

<14961x1 sparse matrix of type '<type 'numpy.int64'>'
	with 14961 stored elements in Compressed Sparse Column format>